In [35]:
from azureml.core import Workspace, Experiment, ScriptRunConfig

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()

Workspace name: obuai-ws-tst
Azure region: westeurope
Subscription id: 38bb9c9d-71c6-4d92-baf3-04f0f8aeaead
Resource group: odap-ai-prd-we-rg


In [36]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = "cpu-cluster-p2"
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_D2_V2"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes,
                                                                idle_seconds_before_scaledown=600)
     
    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

    # For a more detailed view of current AmlCompute status, use get_status()
    # print(compute_target.get_status().serialize())

found compute target: cpu-cluster-p2


## 1. Hyperparameter tuning of custom model with Hyperdrive

In [37]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': loguniform(0.01, 500),
        '--max_iter': choice(10, 50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
from azureml.core import Environment
env = Environment.get(workspace=ws, name='AzureML-Tutorial')
est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      environment=env,
                      compute_target=compute_target)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     run_config=est,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=10)

In [38]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [53]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run_hd = hyperdrive_run.get_best_run_by_primary_metric()
best_run_hd.download_file('outputs/model.joblib', 'trainedmodel/modelhd.joblib')

## 2. AutoML

In [39]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_paths = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_paths)

In [43]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
# clean_data returns two pd.DataFrames but we need a single TabularDataset
x, y = clean_data(ds)
x['y'] = y
pd.DataFrame.to_csv(x, 'training/cleaneddataset.csv')

datastore = ws.get_default_datastore()
datastore.upload(src_dir='training', target_path='training/udacityp1/cleaneddataset.csv', overwrite=True, show_progress=True)

tds = TabularDatasetFactory.from_delimited_files(datastore.path('training/udacityp1/cleaneddataset.csv'))

Uploading an estimated of 1 files
Uploading training/cleaneddataset.csv
Uploaded training/cleaneddataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [46]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=tds,
    label_column_name='y',
    n_cross_validations=2,
    compute_target=compute_target)

In [47]:
# Submit your automl run

### YOUR CODE HERE ###
# Submit model run
automl_run = exp.submit(automl_config, show_output=True)
# Track run in widget
RunDetails(automl_run).show()

Submitting remote run.
No run_configuration provided, running on cpu-cluster-p2 with default configuration
Running on remote compute: cpu-cluster-p2


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_07988e98-c7b6-4493-a47d-64a448f8cf68,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Received interrupt. Returning now.

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [54]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
# Get best child run
best_run = automl_run.get_output()
# Download model from best child run
best_run[0].download_file('outputs/model.pkl', 'trainedmodel/modelautoml.pkl')


Package:azureml-core, training version:1.29.0, current version:1.28.0.post1
Package:azureml-dataset-runtime, training version:1.29.0, current version:1.28.0
Package:azureml-pipeline-core, training version:1.29.0, current version:1.28.0
Package:azureml-telemetry, training version:1.29.0, current version:1.28.0
Package:azureml-train-automl-client, training version:1.29.0, current version:1.28.0
Package:azureml-defaults, training version:1.29.0
Package:azureml-interpret, training version:1.29.0
Package:azureml-mlflow, training version:1.29.0
Package:azureml-model-management-sdk, training version:1.0.1b6.post1
Package:azureml-train-automl-runtime, training version:1.29.0
